<a href="https://colab.research.google.com/github/KorStats/skku/blob/main/%ED%86%B5%ED%95%A9/(%ED%86%B5%ED%95%A9)_EDA_%EA%B3%BC%EC%A0%95_0_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 14.4 MB/s eta 0:00:00


In [2]:
import pickle
import random
from konlpy.tag import Okt

# 동의어 사전 로드
def load_synonym_dict(pickle_file):
    with open(pickle_file, 'rb') as f:
        synonym_dict = pickle.load(f)
    return synonym_dict

# Random Swap (랜덤 단어 교환)
def random_swap(words, n=1):
    words = words.copy()
    for _ in range(n):
        idx1, idx2 = random.sample(range(len(words)), 2)
        words[idx1], words[idx2] = words[idx2], words[idx1]
    return words

# Random Insertion (랜덤 단어 삽입)
def random_insertion(words, synonym_dict, n=1):
    words = words.copy()
    for _ in range(n):
        idx = random.randint(0, len(words) - 1)
        word = words[idx]
        if word in synonym_dict:
            words.insert(idx, random.choice(synonym_dict[word]))
    return words

# Random Delete (랜덤 단어 삭제)
def random_delete(words, p=0.1):
    if len(words) == 1:
        return words
    return [word for word in words if random.random() > p]

# Synonym Replacement (동의어 대체)
def synonym_replacement(words, synonym_dict, n=1):
    words = words.copy()
    for _ in range(n):
        idx = random.randint(0, len(words) - 1)
        word = words[idx]
        if word in synonym_dict:
            words[idx] = random.choice(synonym_dict[word])
    return words

# 데이터 증강 실행 함수
#def augment_text(text, synonym_dict, swap_n=1, insert_n=1, delete_p=0.1, replace_n=1):
#    okt = Okt()
#    words = okt.morphs(text)  # Okt를 이용한 토큰화
#    words = random_swap(words, swap_n)
#    words = random_insertion(words, synonym_dict, insert_n)
#    words = random_delete(words, delete_p)
#    words = synonym_replacement(words, synonym_dict, replace_n)
#    return ' '.join(words)

In [5]:
# 동의어 사전 로드
synonym_dict = load_synonym_dict('/content/wordnet.pickle')

In [6]:
# 데이터 증강 실행 함수
def augment_text(text, synonym_dict, swap_n=0, insert_n=0, delete_p=0, replace_n=0):
    okt = Okt()
    words = okt.morphs(text)  # Okt를 이용한 토큰화

    if swap_n > 0:
        words = random_swap(words, swap_n)
    if insert_n > 0:
        words = random_insertion(words, synonym_dict, insert_n)
    if delete_p > 0:
        words = random_delete(words, delete_p)
    if replace_n > 0:
        words = synonym_replacement(words, synonym_dict, replace_n)

    return ' '.join(words)

# 데이터불러오기

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'Colab Notebooks'   이전   졸업논문


In [12]:
import pandas as pd
df= pd.read_csv('/content/gdrive/MyDrive/졸업논문/NSMC2_0.8_증분대상_1000개.csv', encoding='utf-8-sig')

In [13]:
df.head(3)

,id,document,label
0,9767996,"별 한개도 아깝다...연출, 구성, 시나리오 자체도 완전 폐급...거기다 애들 연기...",0
1,9451342,"예전에 개봉 했을때 영화관 가서 봤었는데 , 그당시에 영화 보다가 잠들었었음 .. ...",0
2,8929135,"평점 이건 아니잖아. 졸라 지루하고, 1시간30분의 러닝타임도 감독이 시간이 남아돌...",0


In [14]:
# 증강 적용
df['text_swap'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=2))
df['text_insert'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=2))
df['text_delete'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.2))
df['text_replace'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=2))

In [15]:
df.to_csv('NSMC2_0.8_aug_eda.csv', encoding='utf-8-sig')